In [1]:
# Verify that you have the GPU recognized
!nvidia-smi

Tue Oct 31 15:59:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install dependencies
!pip uninstall torch torchtext torchvision torchvision torchdata torchaudio
!pip install torch==1.13.0
!pip install transformers==4.24.0
!pip install torchtext==0.14.0
!pip install torchvision==0.14.0
!pip install torchdata==0.5.1
!pip install seqeval==1.2.2

Found existing installation: torch 2.1.0+cu118
Uninstalling torch-2.1.0+cu118:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.1.0+cu118.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? n
Found existing installation: torchtext 0.16.0
Uninstalling torchtext-0.16.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchtext-0.16.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchtext/*
Proceed (Y/n)? n
Found existing installation: torchvision 0.16.0+cu118
Uninstalling torchvision-0.16.0+cu118:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchvision-0.16.0+cu118.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchvision.

In [3]:
# Remove existing package and clone again from Github
!rm -rf /content/ArabicNER
!git clone https://github.com/RamyCodes/ArabicNER.git

Cloning into 'ArabicNER'...
remote: Enumerating objects: 627, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 627 (delta 50), reused 30 (delta 14), pack-reused 526
Receiving objects: 100% (627/627), 387.85 KiB | 4.56 MiB/s, done.
Resolving deltas: 100% (363/363), done.


In [4]:
# Add the ArabicNER package to the system path
import sys
import argparse
sys.path.append('/content/ArabicNER/')

In [5]:
# Import train function
from arabiner.bin.train import main as train

In [6]:
# Setup the model arguments
args_dict = {
    # Model output path to save artifacts and model predictions
    "output_path": "/content/output/",

    # train/test/validation data paths
    "train_path": "/content/ArabicNER/data/train.txt",
    "test_path": "/content/ArabicNER/data/test.txt",
    "val_path": "/content/ArabicNER/data/val.txt",

    # seed for randomization
    "seed": 1,

    "batch_size": 8,

    # Nmber of workers for the dataloader
    "num_workers": 1,

    # GPU/device Ids to train model on
    # For two GPUs use [0, 1]
    # For three GPUs use [0, 1, 2], etc.
    "gpus": [0],

    # Overwrite data in output_path directory specified above
    "overwrite": True,

    # How often to print the logs in terms of number of steps
    "log_interval": 10,

    # Data configuration
    # Here we specify the dataset class and there are two options:
    #  arabiner.data.datasets.DefaultDataset: for flat NER
    #  arabiner.data.datasets.NestedTagsDataset: for nested NER
    #
    # kwargs: keyword arguments to the dataset class
    # This notebook used the DefaultDataset for flat NER
    "data_config": {
        "fn": "arabiner.data.datasets.DefaultDataset",
        "kwargs": {"max_seq_len": 512}
    },

    # Neural net configuration
    # There are two NNs:
    #   arabiner.nn.BertSeqTagger: flat NER tagger
    #   arabiner.nn.BertNestedTagger: nested NER tagger
    #
    # kwargs: keyword arguments to the NN
    # This notebook uses BertSeqTagger for flat NER tagging
    "network_config": {
        "fn": "arabiner.nn.BertSeqTagger",
        "kwargs": {"dropout": 0.1, "bert_model": "aubmindlab/bert-base-arabertv2"}
    },

    # Model trainer configuration
    #
    #  arabiner.trainers.BertTrainer: for flat NER training
    #  arabiner.trainers.BertNestedTrainer: for nested NER training
    #
    # kwargs: keyword arguments to arabiner.trainers.BertTrainer
    #         additional arguments you can pass includes
    #           - clip: for gradient clpping
    #           - patience: number of epochs for early termination
    # This notebook uses BertTrainer for fat NER training
    "trainer_config": {
        "fn": "arabiner.trainers.BertTrainer",
        "kwargs": {"max_epochs": 50}
    },

    # Optimizer configuration
    # Our experiments use torch.optim.AdamW, however, you are free to pass
    # any other optmizers such as torch.optim.Adam or torch.optim.SGD
    # lr: learning rate
    # kwargs: keyword arguments to torch.optim.AdamW or whatever optimizer you use
    #
    # Additional optimizers can be found here:
    # https://pytorch.org/docs/stable/optim.html
    "optimizer": {
        "fn": "torch.optim.AdamW",
        "kwargs": {"lr": 0.0001}
    },

    # Learning rate scheduler configuration
    # You can pass a learning scheduler such as torch.optim.lr_scheduler.StepLR
    # kwargs: keyword arguments to torch.optim.AdamW or whatever scheduler you use
    #
    # Additional schedulers can be found here:
    # https://pytorch.org/docs/stable/optim.html
    "lr_scheduler": {
        "fn": "torch.optim.lr_scheduler.ExponentialLR",
        "kwargs": {"gamma": 1}
    },

    # Loss function configuration
    # We use cross entropy loss
    # kwargs: keyword arguments to torch.nn.CrossEntropyLoss or whatever loss function you use
    "loss": {
        "fn": "torch.nn.CrossEntropyLoss",
        "kwargs": {}
    }
}

# Convert args dictionary to argparse namespace
args = argparse.Namespace()
args.__dict__ = args_dict

In [7]:
# Start training the model
train(args)

Logging to /content/output/train.log
INFO	arabiner.bin.train	Tue, 31 Oct 2023 16:02:48	Writing config to /content/output/args.json


(…)rt-base-arabertv2/resolve/main/vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

(…)tv2/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

(…)ertv2/resolve/main/tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

(…)-base-arabertv2/resolve/main/config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:06	Epoch 0 | Batch 10/78 | Timestep 10 | LR 0.0001000000 | Loss 0.187474
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:08	Epoch 0 | Batch 20/78 | Timestep 20 | LR 0.0001000000 | Loss 0.303947
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:10	Epoch 0 | Batch 30/78 | Timestep 30 | LR 0.0001000000 | Loss 0.183902
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:12	Epoch 0 | Batch 40/78 | Timestep 40 | LR 0.0001000000 | Loss 0.268084
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:14	Epoch 0 | Batch 50/78 | Timestep 50 | LR 0.0001000000 | Loss 0.156375
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:16	Epoch 0 | Batch 60/78 | Timestep 60 | LR 0.0001000000 | Loss 0.357269
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:18	Epoch 0 | Batch 70/78 | Timestep 70 | LR 0.0001000000 | Loss 0.174098
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:20	** Evaluating on valid

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO	root	Tue, 31 Oct 2023 16:03:22	Predictions written to /content/output/predictions.txt
INFO	root	Tue, 31 Oct 2023 16:03:22	
              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000       186
        MISC     0.0000    0.0000    0.0000        93
         ORG     0.0000    0.0000    0.0000       101
        PERS     0.0000    0.0000    0.0000       142

   micro avg     0.0000    0.0000    0.0000       522
   macro avg     0.0000    0.0000    0.0000       522
weighted avg     0.0000    0.0000    0.0000       522

INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:22	Epoch 0 | Timestep 78 | Test Loss 0.346529 | F1 0.000000
INFO	arabiner.trainers.BaseTrainer	Tue, 31 Oct 2023 16:03:22	Saving checkpoint to /content/output/checkpoints/checkpoint_0.pt
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:29	Epoch 1 | Batch 2/78 | Timestep 80 | LR 0.0001000000 | Loss 0.299578
INFO	arabiner.trainers.BertTrainer	Tue, 31 Oct 2023 16:03:31	Ep